In [1]:
# To create interactive plots
import plotly.graph_objects as go
import json
import pandas as pd
import nltk
from os import listdir
from os.path import isfile, join
import seaborn as sns
from tqdm import tqdm
import glob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os
import string
from nltk.stem import WordNetLemmatizer
from pattern.en import tag
from nltk.corpus import wordnet as wn
import warnings
import scipy.sparse as sp 
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
from nltk.corpus import reuters
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import nltk.data
import math
import re
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
import numpy as np
import pandas as pd
import scipy as sc
stop_words = stopwords.words('english')
ideal_sent_length = 20.0
stemmer = SnowballStemmer("english")
import re
from urllib.request import urlopen
#import gensim 
import numpy as np 
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import re
import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim

import spacy
import joblib
from tqdm import tqdm
import seaborn as sb
from matplotlib import pyplot as plt

wnl = WordNetLemmatizer()

In [2]:
import semanticscholar as sch
from IPython.display import display, Latex, HTML, FileLink
from sentence_transformers import SentenceTransformer, models
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

In [3]:
def checkAnyStop(token_list, token_stops):
    return any([stop in token_list for stop in token_stops])

def firstFullStopIdx(token_list, token_stops):
  
    idxs = []
    for stop in token_stops:
        if stop in token_list:
            idxs.append(token_list.index(stop))
    minIdx = min(idxs) if idxs else None
    return minIdx

puncts = ['!', '.', '?', ';']
puncts_tokens = [tokenizer.tokenize(x)[0] for x in puncts]

def splitTokens(tokens, punct_tokens, split_length):
    """
    To avoid splitting a sentence and lose the semantic meaning of it, a paper is splitted 
    into chunks in such a way that each chunk ends with a full-stop token (['.' ';' '?' or '!']) 
    """
    splitted_tokens = []
    while len(tokens) > 0:
        if len(tokens) < split_length or not checkAnyStop(tokens, punct_tokens):
            splitted_tokens.append(tokens)
            break
        # to not have too long parapraphs, the nearest fullstop is searched both in the previous 
        # and the next strings.
        prev_stop_idx = firstFullStopIdx(tokens[:split_length][::-1], puncts_tokens)
        next_stop_idx = firstFullStopIdx(tokens[split_length:], puncts_tokens)
        if pd.isna(next_stop_idx):
            splitted_tokens.append(tokens[:split_length - prev_stop_idx])
            tokens = tokens[split_length - prev_stop_idx:]
        elif pd.isna(prev_stop_idx):
            splitted_tokens.append(tokens[:split_length + next_stop_idx + 1])
            tokens = tokens[split_length + next_stop_idx + 1:] 
        elif prev_stop_idx < next_stop_idx:
            splitted_tokens.append(tokens[:split_length - prev_stop_idx])
            tokens = tokens[split_length - prev_stop_idx:]
        else:
            splitted_tokens.append(tokens[:split_length + next_stop_idx + 1])
            tokens = tokens[split_length + next_stop_idx + 1:] 
    return splitted_tokens

def splitParagraph(text, split_length=90):
    tokens = tokenizer.tokenize(text)
    splitted_tokens = splitTokens(tokens, puncts_tokens, split_length)
    return [tokenizer.convert_tokens_to_string(x) for x in splitted_tokens]

In [4]:
res=pd.read_csv('sabrina.csv')

In [5]:
SUMMARY_LENGTH = 12  # number of sentences in final summary
stop_words = stopwords.words('english')


In [7]:
class Summarizer():

    def penn_to_wn(self,tag):
        
        if tag.startswith('N'):
            return 'n'
 
        if tag.startswith('V'):
        
            return 'v'
 
        if tag.startswith('J'):
            return 'a'
 
        if tag.startswith('R'):
            return 'r'
        return None 
 

    def tagged_to_synset(self,word, tag):
        wn_tag = self.penn_to_wn(tag)
    
        if wn_tag is None:
            return None
 
        try:
        
            return wn.synsets(word, wn_tag)[0]
        except:
            return None
 
    def sentence_similarity(self,sentence1, sentence2):
       
        # Tokenize and tag
        sentence1 = pos_tag(word_tokenize(sentence1))
        
        sentence2 = pos_tag(word_tokenize(sentence2))
      
 
        # Get the synsets for the tagged words
        synsets1 = [self.tagged_to_synset(*tagged_word) for tagged_word in sentence1]
        
        synsets2 = [self.tagged_to_synset(*tagged_word) for tagged_word in sentence2]
       
 
    # Filter out the Nones
        synsets1 = [ss for ss in synsets1 if ss]
      
        synsets2 = [ss for ss in synsets2 if ss]
        
 
        score, count = 0, 0
        for synset in synsets1:
            max_sim = 0.0
            maxx=0
            for ss in synsets2:
                
                sim=wn.path_similarity(synset, ss)
                if sim is not None and sim > max_sim:
                
                       max_sim = sim
                   
           
        
            if max_sim is not None and max_sim!=0:
                score += max_sim
                count += 1
 
    # Average the values
        if count!=0:
            score /= count
            return score   
    def __init__(self, article):
            self._articles = []
            i=1
           
            for row in article:
                #if i<=5:
                    title=row[3]
                    #print(title)
                    body=row[17].replace('\n', ' ')
                    #print(body)
                    paper_id=row[0]
                    doi=row[5]
                    if title!=''and body!='':
                        self._articles.append((paper_id,title,body,doi))
                 #   i=i+1
            
            
    def valid_input(self, headline, article_text):
        return headline != '' and article_text != ''    
    def normalize_corpus(self,corpus, lemmatize=True):
        
        normalized_corpus = []    
        for text in corpus:
            if lemmatize:
                text = self.lemmatize_text(text)
            else:
                text = text.lower()
            text = self.remove_special_characters(text)
            text = self.remove_stopwords(text)
            normalized_corpus.append(text)
        return normalized_corpus
    def pos_tag_text(self,text):
        wnl = WordNetLemmatizer()
        tagged_text = tag(text)
        tagged_lower_text = [(word.lower(), self.penn_to_wn(pos_tag))
                             for word, pos_tag in
                             tagged_text]
        return tagged_lower_text
        
         
    def score(self,article,query):
        """ Assign each sentence in the document a score"""
        Query=[]
        Query.append(query)
        headline = article[1]
        #sentences = self.split_into_sentences(article[2])
        sentences=splitParagraph(article[2])
        norm_corpus =self.normalize_corpus(sentences, lemmatize=True)
        #print(norm_corpus)
        # normalize model_answer
        norm_model_answer =  self.normalize_corpus(Query, lemmatize=True)   
        vectorizer, corpus_features = self.build_feature_matrix(norm_corpus,feature_type='frequency')
        # extract features from model_answer
        model_answer_features = vectorizer.transform(norm_model_answer)

        doc_lengths = [len(doc.split()) for doc in norm_corpus]   
        avg_dl = np.average(doc_lengths) 
        corpus_term_idfs = self.compute_corpus_term_idfs(corpus_features, norm_corpus)
        for index, doc in enumerate(Query):

            doc_features = model_answer_features[index]
            self.bm25_scores = self.compute_bm25_similarity(doc_features,corpus_features,doc_lengths,avg_dl,corpus_term_idfs,k1=1.5, b=0.75)
            self.semantic_similarity_scores=[]
        for i, s in enumerate(sentences):
                score1=self.sentence_similarity(s,query)
                score2=self.sentence_similarity(query,s)
                if score1 is not None and score2 is not None:
                    score=(score1+score2)/2
                    self.semantic_similarity_scores.append(score)
                elif score1 is not None and score2 is None:
                    self.semantic_similarity_scores.append(score1)
                elif score2 is not None and score1 is None:
                    self.semantic_similarity_scores.append(score2)
        doc_index=0
        sim_score=[]
        for score_tuple in zip(self.semantic_similarity_scores,self.bm25_scores):
            sim_score.append(((score_tuple[0])+score_tuple[1]/10)/2)
        for tuple_ in zip(sentences,sim_score):
            s=tuple_[0]
            self._scores[s]=tuple_[1]
        
        
                    
              
    def generate_summaries(self):
        
            self.dict_ = {'task':[],'doi':[],'paper_id':[],'title':[],'summary': [],'score':[],'sentences':[]}
            jj=1
            ii=1
          
            tasks=['what is the immune system response to 2019-ncov ?'
                   
                   ]
           
    
            for query in tasks:
                for article in self._articles:
                    self._scores = Counter()
                    self.score(article,query)
                    highest_scoring = self._scores.most_common(SUMMARY_LENGTH)
                    #totalsentences = splitParagraph(article[2])
                    #summarylist=[]
                    #summr=[sent[0] for sent[0] in highest_scoring]
                   
                    #for sentence in totalsentences:
                     #   for sumsen in  summr:
                      #      if sentence==sumsen:
                       #         summarylist.append(sentence)        
                    # Appends highest scoring "representative" sentences, returns as a single summary paragraph.
                    summary=' '.join([sent[0] for sent in highest_scoring])
                    s=0
                    for scr in highest_scoring:
                        s=s+scr[1]
                    s=s/12    
                    
                    
                    ''' 
                    print('**task**')
                    print(query)
                    print(" **Title: **")
                    print(article[1])
                    print(highest_scoring)
                    print('**summary**')
                    print(summary)
                    print('____________________________________________________________________________________')
                    '''   
                    
                    self.dict_['sentences'].append(highest_scoring)
                    self.dict_['summary'].append(summary)
                    self.dict_['title'].append(article[1])
                    self.dict_['paper_id'].append(article[0])
                    self.dict_['task'].append(query)
                    self.dict_['doi'].append(article[3])
                    self.dict_['score'].append(s)
            self.papers = pd.DataFrame(self.dict_, columns=['task','paper_id','doi','title','summary','score','sentences'])
            return self.papers
       

    def remove_smart_quotes(self, text):
       
       
        #text=re.sub("([\(\[].*?[\)\]][\(\[].*?[\)\]]+)", ' ', text)
        text=re.sub("[\(\[].*?[\)\]]", '', text)
        
        try:
            url = re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', text)
            repl_url = url.group(3)
            text = re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)',repl_url, text)
        except:
            pass #there might be emails with no url in them
        #text=re.sub("[\[]()*?[\]]", "", text)#remove in-text citation
        
        text=re.sub(r"[<>()(,)|&©ø\[\]\'\";?~*!]", ' ', text) #remove <>()|&©ø"',;?~*!
        text=re.sub("(\\t)", ' ', text) #remove escape charecters
        text=re.sub("(\\r)", ' ', text) 
        text=re.sub("(\\n)", ' ', text)
        text= re.sub("(\s+)",' ',text) #remove multiple space
        text=re.sub("doi", ' ',text)
        
        return text


    def split_into_sentences(self, text):
        tok = nltk.data.load('tokenizers/punkt/english.pickle')
        sentences = tok.tokenize(text)
        sentences = [sent.replace('\n', '') for sent in sentences if len(sent) > 20]   
    
        return sentences
    def lemmatize_text(self,text):
    
        pos_tagged_text = self.pos_tag_text(text)
        lemmatized_tokens = [wnl.lemmatize(word, pos_tag) if pos_tag
                             else word                     
                             for word, pos_tag in pos_tagged_text]
        lemmatized_text = ' '.join(lemmatized_tokens)
        return lemmatized_text

    def remove_special_characters(self,text):
        tokens = self.tokenize_text(text)
        pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
        filtered_tokens = filter(None, [pattern.sub(' ', token) for token in tokens])
        filtered_text = ' '.join(filtered_tokens)
        return filtered_text

    def remove_stopwords(self,text):
        stopword_list = nltk.corpus.stopwords.words('english')
        tokens = self.tokenize_text(text)
        filtered_tokens = [token for token in tokens if token not in stopword_list]
        filtered_text = ' '.join(filtered_tokens)    
        return filtered_text

    
  
    def build_feature_matrix(self,documents, feature_type='frequency',
                         ngram_range=(1, 1), min_df=0.0, max_df=1.0):

        feature_type = feature_type.lower().strip()  
    
        if feature_type == 'binary':
            vectorizer = CountVectorizer(binary=True, min_df=min_df,
                                     max_df=max_df, ngram_range=ngram_range)
        elif feature_type == 'frequency':
            vectorizer = CountVectorizer(binary=False, min_df=min_df,
                                     max_df=max_df, ngram_range=ngram_range)
        elif feature_type == 'tfidf':
            vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df, 
                                     ngram_range=ngram_range)
        else:
            raise Exception("Wrong feature type entered. Possible values: 'binary', 'frequency', 'tfidf'")

        feature_matrix = vectorizer.fit_transform(documents).astype(float)
    
        return vectorizer, feature_matrix

    def compute_corpus_term_idfs(self,corpus_features, norm_corpus):
    
        dfs = np.diff(sp.csc_matrix(corpus_features, copy=True).indptr)
        dfs = 1 + dfs # to smoothen idf later
        total_docs = 1 + len(norm_corpus)
        idfs = 1.0 + np.log(float(total_docs) / dfs)
        return idfs
    def compute_bm25_similarity(self,doc_features, corpus_features,
                            corpus_doc_lengths, avg_doc_length,
                            term_idfs, k1=1.5, b=0.75):
    # get corpus bag of words features
        corpus_features = corpus_features.toarray()
    # convert query document features to binary features
    # this is to keep a note of which terms exist per document
        doc_features = doc_features.toarray()[0]
        doc_features[doc_features >= 1] = 1
    
    # compute the document idf scores for present terms
        doc_idfs = doc_features * term_idfs
    # compute numerator expression in BM25 equation
        numerator_coeff = corpus_features * (k1 + 1)
        numerator = np.multiply(doc_idfs, numerator_coeff)
    # compute denominator expression in BM25 equation
        denominator_coeff =  k1 * (1 - b + 
                                (b * (corpus_doc_lengths / 
                                        avg_doc_length)))
        denominator_coeff = np.vstack(denominator_coeff)
        denominator = corpus_features + denominator_coeff
    # compute the BM25 score combining the above equations
        bm25_scores = np.sum(np.divide(numerator,
                                   denominator),
                         axis=1) 
    
        return bm25_scores
    def tokenize_text(self,text):
        tokens = nltk.word_tokenize(text) 
        tokens = [token.strip() for token in tokens]
        return tokens

    

In [8]:
import csv
import nltk
csv.field_size_limit(100000000)
results=pd.DataFrame()
f= open("sabrina.csv", encoding="utf-8-sig")
reader = csv.reader(f, delimiter=',')
next(reader)
summaries= Summarizer(reader)
results=summaries.generate_summaries()
resultss = results.sort_values(by='score', ascending=False)
resultss.head(50)

,task,paper_id,doi,title,summary,score,sentences
822,what is the immune system response to 2019-ncov ?,40dabecefc2f0a2b750e000d1c15cca57a3f7dd4,10.1016/0163-7258(85)90033-6,ANIMAL VIRUSES OF ECONOMIC IMPORTANCE: GENETIC...,it is convenient to divide the immune system i...,0.896358,[(it is convenient to divide the immune system...
759,what is the immune system response to 2019-ncov ?,2008f024274c53b0de1276f4047a6a641813fd98,10.1016/S1081-1206(10)61294-X,ABSTRACTS: CONCURRENT SESSIONS,common variable immunodeficiency cvid is the m...,0.800850,[(common variable immunodeficiency cvid is the...
1232,what is the immune system response to 2019-ncov ?,fea50730d7cdcd7c728f98c31fe20aeb8af5e350,10.1016/j.compbiomed.2020.103670,Computers and viral diseases. Preliminary bioi...,it is to be understood that peptides proposed ...,0.744558,[(it is to be understood that peptides propose...
943,what is the immune system response to 2019-ncov ?,78e39a957cc704ddc919e704459258e55178f574,10.1016/j.prrv.2008.05.005,Innate immune response and bronchiolitis and p...,although the association between bronchiolitis...,0.689573,[(although the association between bronchiolit...
904,what is the immune system response to 2019-ncov ?,6be4fe1605ea1a6f558204da253019a399be3178,10.1016/B978-0-12-385885-6.00009-2,Coronavirus Pathogenesis,about 40 % of these noncoding rnas are similar...,0.678121,[(about 40 % of these noncoding rnas are simil...
622,what is the immune system response to 2019-ncov ?,b30770ae30b35cdfaf0a173863e74e93edbb0329,10.1186/s13054-016-1208-6,P001 Sepsis impairs the capillary response wit...,conclusions : this study demonstrates that cyt...,0.672039,[(conclusions : this study demonstrates that c...
902,what is the immune system response to 2019-ncov ?,6aadca94314fe7e81e278fbbef178f2a5bf4f538,10.1186/cc13531,Minimising prescribing errors in the ICU DJ Me...,interleukin - 8 il - 8 a key modulator of neut...,0.660494,[(interleukin - 8 il - 8 a key modulator of ne...
898,what is the immune system response to 2019-ncov ?,68ac63121bded12e1db3178a0c9b050154f81ab8,10.1016/j.mehy.2015.11.005,Seasonality and selective trends in viral acut...,an interesting and ingenious recent review loo...,0.652632,[(an interesting and ingenious recent review l...
777,what is the immune system response to 2019-ncov ?,2c66349f1922022b585ae0e5d6c0776c27d2852d,10.1016/B978-012369454-6/50034-0,Chapter 6 Mouse Hepatitis Virus,mhv and neurotropic mhv - jhm in particular or...,0.652208,[(mhv and neurotropic mhv - jhm in particular ...
765,what is the immune system response to 2019-ncov ?,2437ecdfe29ce5195079af809de646e9e1e1350a,10.1016/B978-012373944-5.00314-X,Respiratory Viruses,however this protection is limited and reinfec...,0.639031,[(however this protection is limited and reinf...


In [13]:
def postprocessing(text):
    # capitalize the text
    text = text.capitalize()
    # '2 , 3' -> '2,3'
    text = re.sub(r'(\d) \, (\d)', "\g<1>,\g<2>", text)
    # full stop
    text = re.sub(r' (\.|\!|\?) (\w)', lambda pat: pat.groups()[0]+" "+pat.groups()[1].upper(), text)
    # full stop at the end
    text = re.sub(r' (\.|\!|\?)$', "\g<1>", text)
    # comma
    text = re.sub(r' (\,|\;|\:) (\w)', "\g<1> \g<2>", text)
    # - | \ / @ and _ (e.g. 2 - 3  -> 2-3)
    text = re.sub(r'(\w) (\-|\||\/|\\|\@\_) (\w)', "\g<1>\g<2>\g<3>", text)
    # parenthesis
    text = re.sub(r'(\(|\[|\{) ([^\(\)]+) (\)|\]|\})', "\g<1>\g<2>\g<3>", text)
    # "" e.g. " word "  -> "word"
    text = re.sub(r'(\") ([^\"]+) (\")', "\g<1>\g<2>\g<3>", text)
    # apostrophe  e.g. l ' a  -> l'a
    text = re.sub(r'(\w) (\') (\w)', "\g<1>\g<2>\g<3>", text)
    # '3 %' ->  '3%'
    text = re.sub(r'(\d) \%', "\g<1>%", text)
    # '# word'  -> '#word'
    text = re.sub(r'\# (\w)', "#\g<1>", text)
    # https and doi
    text = re.sub(r'(https|doi) : ', "\g<1>:", text)
    return text

In [9]:
resultss.to_csv('paragraphs.csv', index=False)


In [10]:
corpus=[]
scores = Counter()

for i in range(len(resultss)):
    
        
        for x in resultss.sentences[i]:
        
            corpus.append(x)



In [11]:
scores = Counter()
for item in corpus:
    #print(item[0])
    s=item[0]
    scores[s]=item[1]
    

    

In [12]:
highest_scoring =scores.most_common(20)
                    
summr=[sent[0] for sent in highest_scoring]
                   

summary=' '.join([sent for sent in  summr])
print(summary)

the final set of immune vips is made of the 203 vips annotated with the go categories immune system process go : 0002376 defense response go : 0006952 or immune response go : 0006955 2015 in addition to 38 immune vips we annotated based on publications reporting roles in various parts of the immune response from innate to adaptive immune response from regulators to effectors of the immune response to vips involved in the development of the immune system . therefore spike determines to some extent the host range . potential anti - coronavirus therapies can be divided into two categories depending on the target one is acting on the human immune system or human cells and the other is on coronavirus itself . in terms of the human immune system the innate immune system response plays an important role in controlling the replication and infection of coronavirus and interferon is expected to enhance the immune response 11 . while strong cellular immune responses may control the disease peders

In [14]:
postprocessing(summary)

'The final set of immune vips is made of the 203 vips annotated with the go categories immune system process go: 0002376 defense response go: 0006952 or immune response go: 0006955 2015 in addition to 38 immune vips we annotated based on publications reporting roles in various parts of the immune response from innate to adaptive immune response from regulators to effectors of the immune response to vips involved in the development of the immune system. Therefore spike determines to some extent the host range. Potential anti-coronavirus therapies can be divided into two categories depending on the target one is acting on the human immune system or human cells and the other is on coronavirus itself. In terms of the human immune system the innate immune system response plays an important role in controlling the replication and infection of coronavirus and interferon is expected to enhance the immune response 11. While strong cellular immune responses may control the disease pedersen 2009 

In [18]:
from fpdf import FPDF 
  
pdf = FPDF() 
pdf.add_page() 

pdf.set_font("Arial", size = 15) 
  

  
# add another cell 
pdf.cell(200, 10, txt = "Top paragraphs.", 
         ln = 1, align = 'C') 
pdf.cell(200, 10, txt = 'gyjygkihuk', 
         ln = 1, align = 'C') 
  
# save the pdf with name .pdf 
pdf.output("top paragraphs.pdf")    

''

In [19]:
import tkinter as tk